In [1]:
import torch
import torch.nn as nn
import torchvision

print(torch.__version__)
print(torch.version.cuda)
print(torch.backends.cudnn.version())
print(torch.cuda.is_available())#输出false，表示当前pytorch无法使用显卡
# https://zhuanlan.zhihu.com/p/419063125

1.11.0+cpu
None
None
False


In [2]:
# 在同一个设备上，应该保证可复现性。具体做法是，在程序开始的时候固定torch的随机种子，同时也把numpy的随机种子固定。
import numpy as np

np.random.seed(0)
torch.manual_seed(0)#设置随机数种子
torch.cuda.manual_seed_all(0)#多个GPU时，为所有GPU设置种子

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
# 显卡设置
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'
# torch.cuda.empty_cache()

cpu


In [4]:
#张量处理
tensor = torch.randn(3,4,5)
print(tensor.type())
print(tensor.size())
print(tensor.dim())

torch.FloatTensor
torch.Size([3, 4, 5])
3


In [5]:
# 张量命名
NCHW = ['N','C','H','W']
images = torch.randn(32, 3, 56, 56, names=NCHW)
images.sum('C')
images.select('C', index=0)

C:\Users\20114\AppData\Local\Temp\ipykernel_95648\2731278148.py:3: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10/core/TensorImpl.h:1463.)
  images = torch.randn(32, 3, 56, 56, names=NCHW)


tensor([[[-1.1256e+00, -3.1700e-01, -1.0925e+00,  ..., -4.1891e-01,
          -8.0483e-01,  5.6561e-01],
         [ 1.8037e-01,  1.0833e-01, -7.5482e-01,  ..., -7.2091e-01,
           1.4708e+00,  2.7564e-01],
         [ 1.1025e-01, -8.1881e-01,  6.3277e-01,  ..., -1.7557e+00,
           7.6166e-02, -1.0786e+00],
         ...,
         [ 6.3545e-01,  3.0746e-01, -1.2414e+00,  ..., -1.8328e+00,
           8.8194e-01, -1.3492e+00],
         [-2.0224e-01,  4.5614e-01,  3.7426e-01,  ..., -2.2916e-01,
          -2.3076e-01, -4.9349e-01],
         [-2.3604e-01, -1.6773e-01,  4.4987e-01,  ..., -8.7382e-01,
          -4.8728e-01, -1.6210e-01]],

        [[-5.8514e-01,  3.1384e-02, -2.4608e-02,  ...,  5.7994e-01,
          -3.6082e-01,  3.1634e-01],
         [ 4.7955e-01,  8.3371e-01,  8.7849e-01,  ..., -5.5736e-01,
           1.4210e+00,  1.4998e-01],
         [-3.0390e-01, -8.2490e-01, -1.6068e+00,  ...,  3.1689e-02,
           2.3211e+00,  1.1986e-01],
         ...,
         [ 8.3218e-01, -4

In [6]:
# 设置默认类型，pytorch中的FloatTensor远远快于DoubleTensor
torch.set_default_tensor_type(torch.FloatTensor)

# 类型转换
# tensor = tensor.cuda()
tensor = tensor.cpu()
# tensor = tensor.float()
# tensor = tensor.long()

In [7]:
# tensor和ndarray互换
# tensor用逗号隔开，ndarray用空格隔开
ndarray = tensor.cpu().numpy()
tensor = torch.from_numpy(ndarray).float()
print(ndarray,'\n',tensor)

[[[-1.1258398  -1.1523602  -0.25057858 -0.4338788   0.84871036]
  [ 0.69200915 -0.31601277 -2.1152194   0.32227492 -1.2633348 ]
  [ 0.3499832   0.30813393  0.11984151  1.2376579   1.1167772 ]
  [-0.24727815 -1.3526537  -1.6959312   0.5666506   0.79350835]]

 [[ 0.59883946 -1.5550951  -0.3413604   1.8530061   0.7501895 ]
  [-0.58549756 -0.17339675  0.18347794  1.3893661   1.5863342 ]
  [ 0.94629836 -0.84367675 -0.6135831   0.03159274 -0.49267697]
  [ 0.24841475  0.43969584  0.11241119  0.64079237  0.44115627]]

 [[-0.10230965  0.792444   -0.2896677   0.05250749  0.59432304]
  [ 1.5419426   0.5073344  -0.59103316 -0.5692481   0.9199714 ]
  [ 1.1108161   1.2898741  -1.4959149  -0.19383712  0.4455122 ]
  [ 1.3252748  -1.629326   -0.54974365 -0.47983426 -0.49968153]]] 
 tensor([[[-1.1258, -1.1524, -0.2506, -0.4339,  0.8487],
         [ 0.6920, -0.3160, -2.1152,  0.3223, -1.2633],
         [ 0.3500,  0.3081,  0.1198,  1.2377,  1.1168],
         [-0.2473, -1.3527, -1.6959,  0.5667,  0.7935]],

In [8]:
print(tensor[1,2,3])#取近似数
tensor[1,2,3].item()#取精确值

tensor(0.0316)


0.03159274160861969

In [9]:
# 张量连续性stride[i]=stride[i+1]*size[i+1]
# 相比torch.view，torch.reshape可以自动处理输入张量不连续的情况。
tensor = torch.rand(2,3,4)
print(tensor.size())
print(tensor.stride())
shape = (6, 4)
tensor = torch.reshape(tensor, shape)
print(tensor.size())
print(tensor.stride())

torch.Size([2, 3, 4])
(12, 4, 1)
torch.Size([6, 4])
(4, 1)


In [10]:
# 打乱顺序
tensor = tensor[torch.randperm(tensor.size(0))]  # 打乱第一个维度
# 水平翻转
# pytorch不支持tensor[::-1]这样的负步长操作，水平翻转可以通过张量索引实现
# 假设张量的维度为[N, D, H, W].
tensor = tensor[:,:,:,torch.arange(tensor.size(3) - 1, -1, -1).long()]

IndexError: Dimension out of range (expected to be in range of [-2, 1], but got 3)

In [ ]:
# 复制张量
# Operation                 |  New/Shared memory | Still in computation graph |
tensor.clone()            # |        New         |          Yes               |
# tensor.detach()           # |      Shared        |          No                |
# tensor.detach.clone()()   # |        New         |          No                |

tensor([[0.8581, 0.2416, 0.9882, 0.9786],
        [0.2575, 0.9111, 0.1800, 0.5429],
        [0.7187, 0.2454, 0.1468, 0.2387],
        [0.4948, 0.3149, 0.1048, 0.6617],
        [0.8109, 0.9278, 0.0064, 0.6038],
        [0.1925, 0.2042, 0.1999, 0.2708]])

In [ ]:
tensor1 = torch.rand(2,3)
tensor2 = torch.rand(2,3)
tensor3 = torch.rand(2,3)

'''
注意torch.cat和torch.stack的区别在于torch.cat沿着给定的维度拼接，
而torch.stack会新增一维。例如当参数是3个10x5的张量，torch.cat的结果是30x5的张量，
而torch.stack的结果是3x10x5的张量。
'''
# print(torch.cat([tensor1,tensor2,tensor3], dim=0))
print(torch.stack([tensor1,tensor2,tensor3], dim=0))

tensor([[[0.5324, 0.2848, 0.2947],
         [0.7555, 0.3111, 0.3616]],

        [[0.0018, 0.7423, 0.2953],
         [0.6169, 0.5384, 0.1215]],

        [[0.6264, 0.4865, 0.1553],
         [0.2721, 0.2186, 0.2387]]])


In [ ]:
# 整数转one-hot
# pytorch的标记默认从0开始
tensor = torch.tensor([0, 2, 1, 3])
N = tensor.size(0)
num_classes = 4
one_hot = torch.zeros(N, num_classes).long()# 输出4*4全零
one_hot.scatter_(dim=1, index=torch.unsqueeze(tensor, dim=1), src=torch.ones(N, num_classes).long())

In [13]:
# 得到非零元素坐标
tensor = torch.tensor([0, 2, 1, 3])
torch.nonzero(tensor)

tensor([[1],
        [2],
        [3]])

In [17]:
# 判断两个张量相等
tensor1 = torch.tensor([0.0, 2.1, 1, 3])
tensor2 = torch.tensor([0.0, 2.1, 1, 3])
# torch.allclose(tensor1,tensor2)
torch.equal(tensor1,tensor2)

True

In [18]:
# 张量扩展
tensor = torch.rand(64,512)
torch.reshape(tensor, (64, 512, 1, 1)).expand(64, 512, 7, 7)

tensor([[[[0.8170, 0.8170, 0.8170,  ..., 0.8170, 0.8170, 0.8170],
          [0.8170, 0.8170, 0.8170,  ..., 0.8170, 0.8170, 0.8170],
          [0.8170, 0.8170, 0.8170,  ..., 0.8170, 0.8170, 0.8170],
          ...,
          [0.8170, 0.8170, 0.8170,  ..., 0.8170, 0.8170, 0.8170],
          [0.8170, 0.8170, 0.8170,  ..., 0.8170, 0.8170, 0.8170],
          [0.8170, 0.8170, 0.8170,  ..., 0.8170, 0.8170, 0.8170]],

         [[0.8487, 0.8487, 0.8487,  ..., 0.8487, 0.8487, 0.8487],
          [0.8487, 0.8487, 0.8487,  ..., 0.8487, 0.8487, 0.8487],
          [0.8487, 0.8487, 0.8487,  ..., 0.8487, 0.8487, 0.8487],
          ...,
          [0.8487, 0.8487, 0.8487,  ..., 0.8487, 0.8487, 0.8487],
          [0.8487, 0.8487, 0.8487,  ..., 0.8487, 0.8487, 0.8487],
          [0.8487, 0.8487, 0.8487,  ..., 0.8487, 0.8487, 0.8487]],

         [[0.0250, 0.0250, 0.0250,  ..., 0.0250, 0.0250, 0.0250],
          [0.0250, 0.0250, 0.0250,  ..., 0.0250, 0.0250, 0.0250],
          [0.0250, 0.0250, 0.0250,  ..., 0

In [ ]:
# 矩阵乘法: (m*n) * (n*p) * -> (m*p).
result = torch.mm(tensor1, tensor2)

# 批矩阵乘法: (b*m*n) * (b*n*p) -> (b*m*p)
result = torch.bmm(tensor1, tensor2)

# 元素相乘
result = tensor1 * tensor2

In [2]:
# 计算两组数据之间的两两欧式距离
import torch
X1 = torch.randn(4, 6)
X2 = torch.randn(4, 6)
dist = torch.sqrt(torch.sum((X1[:,None,:] - X2) ** 2, dim=2))
print(dist)

tensor([[4.4349, 4.7255, 2.7594, 4.3583],
        [3.0444, 2.9798, 2.6099, 2.9358],
        [2.8670, 4.1039, 3.0274, 3.5850],
        [2.9068, 2.3100, 2.4597, 3.7301]])
